In [1]:
import pandas as pd
from sqlalchemy import create_engine
from tqdm.auto import tqdm
import xml.etree.ElementTree as elemTree

In [2]:
csv_paths = [ 
  ('069500', r'..\data\kodex_200.csv'),
  ('114800', r'..\data\kodex_inverse.csv'),
  ('226490', r'..\data\kodex_kospi.csv'),
  ('001', r'..\data\kospi.csv'),
  ('201', r'..\data\kospi200.csv')
]

In [3]:
dfs = []
for st_code, csv_path in tqdm(csv_paths):
  df = pd.read_csv(csv_path, dtype={'체결시간':str})[['체결시간', '시가', '고가', '저가', '현재가', '거래량']]
  df.columns = ['dt', 'open', 'high', 'low', 'close', 'volume']
  # sqlite3에서는 datetime을 지원하지 않으므로, str로 유지한다.
  # df['dt'] = pd.to_datetime(df['dt'], format='%Y%m%d%H%M%S')
  df['st_code'] = st_code
  for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = df[col].abs()
  dfs.append(df)
whole_df = pd.concat(dfs, ignore_index=True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
tree = elemTree.parse(r'../config/.config.xml')
root = tree.getroot()
node_sqlite3 = root.find('./DBMS/sqlite3')
config_db = {tag:node_sqlite3.find(tag).text for tag in ['database']}

In [7]:
config_db

{'database': '../db/kiwoom_db.sqlite3'}

In [8]:
db_engine = create_engine(f'sqlite:///{config_db["database"]}', echo=False)

In [9]:
db_engine.execute('DROP TABLE IF EXISTS data_in_minute')

In [10]:
table_query = '''
CREATE TABLE data_in_minute (
  st_code TEXT not NULL,
  dt TEXT not NULL,
  open INTEGER,
  high INTEGER,
  low INTEGER,
  close INTEGER,
  volume INTEGER,
  PRIMARY KEY (st_code, dt)
)
'''

In [11]:
db_engine.execute(table_query)

## SqlAlchemy + sqlite3 + pd.DataFrame.to_sql()

5.2s (4.8s, 5.1s, 5.4s)

In [12]:
whole_df.to_sql('data_in_minute', db_engine, if_exists='append', index=False)

467847

In [ ]:
db_engine.execute("select * from data_in_minute where st_code='069500' and dt >= '20220601000000'").fetchall()